<a href="https://colab.research.google.com/github/Soham109/Headstarter_Residence-Bonus_Project/blob/main/Headstarter_Bonus_Project_%5B2%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier
!pip install category_encoders
from category_encoders import BinaryEncoder

In [2]:
train_df = pd.read_csv('fraudTrain.csv')
test_df = pd.read_csv('fraudTest.csv')

In [3]:
target = train_df['is_fraud']
features = train_df.drop(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'category', 'first', 'last',
                          'dob', 'trans_num', 'zip', 'unix_time','is_fraud'], axis=1)

In [4]:
target_test = test_df['is_fraud']
features_test = test_df.drop(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'category', 'first', 'last',
                          'dob', 'trans_num', 'zip', 'unix_time','is_fraud'], axis=1)

In [5]:
encoder = BinaryEncoder(cols=['gender', 'state', 'merchant','job','city', 'street'])
features_test_encoded = encoder.fit_transform(features_test)
features_encoded = encoder.fit_transform(features)

In [ ]:
features_encoded

,merchant_0,merchant_1,merchant_2,merchant_3,merchant_4,merchant_5,merchant_6,merchant_7,merchant_8,merchant_9,...,job_1,job_2,job_3,job_4,job_5,job_6,job_7,job_8,merch_lat,merch_long
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,36.011293,-82.048315
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,49.159047,-118.186462
2,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,1,1,43.150704,-112.154481
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,47.034331,-112.561071
4,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,1,38.674999,-78.632459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,0,0,1,0,1,1,1,1,1,1,...,0,1,0,0,1,1,0,0,36.841266,-111.690765
1296671,1,0,1,0,0,0,0,1,1,0,...,1,1,1,1,1,0,1,1,38.906881,-78.246528
1296672,0,1,1,0,0,1,0,1,1,0,...,0,0,0,0,1,1,1,1,33.619513,-105.130529
1296673,1,0,0,0,0,0,0,0,1,0,...,1,1,0,0,0,0,1,1,42.788940,-103.241160


In [ ]:
features_test_encoded

,merchant_0,merchant_1,merchant_2,merchant_3,merchant_4,merchant_5,merchant_6,merchant_7,merchant_8,merchant_9,...,job_1,job_2,job_3,job_4,job_5,job_6,job_7,job_8,merch_lat,merch_long
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,33.986391,-81.200714
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,39.450498,-109.960431
2,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,1,1,40.495810,-74.196111
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,28.812398,-80.883061
4,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,1,44.959148,-85.884734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,0,1,0,0,1,0,1,0,0,1,...,0,0,0,1,0,0,0,1,39.946837,-91.333331
555715,0,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,29.661049,-96.186633
555716,0,0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,1,1,1,46.658340,-119.715054
555717,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,1,1,1,44.470525,-117.080888


In [6]:
features_test_encoded['PoptoAmtRatio'] = features_test_encoded['amt'] / features_test_encoded['city_pop']
features_encoded['PoptoAmtRatio'] = features_encoded['amt'] / features_encoded['city_pop']

In [7]:
X_train = features_encoded
y_train = target
X_test = features_test_encoded
y_test = target_test

In [8]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
def evaluate_and_save_model(model, X_train, X_test, y_train, y_test, filename):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model.__class__.__name__} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, predictions))
    print("-------------")

    with open(filename, "wb") as file:
        pickle.dump(model, file)

    print(f"Model saved as {filename}")

In [ ]:
lr_model = LogisticRegression(random_state=42)
evaluate_and_save_model(lr_model, X_train, X_test, y_train, y_test, 'lr_model.pkl')

LogisticRegression Accuracy: 0.9955
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.00      0.00      0.00      2145

    accuracy                           1.00    555719
   macro avg       0.50      0.50      0.50    555719
weighted avg       0.99      1.00      0.99    555719

-------------
Model saved as lr_model.pkl


In [ ]:
xgb_model = xgb.XGBClassifier(random_state=42)
evaluate_and_save_model(xgb_model, X_train, X_test, y_train, y_test, 'xgb_model.pkl')

XGBClassifier Accuracy: 0.9951
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.28      0.17      0.21      2145

    accuracy                           1.00    555719
   macro avg       0.64      0.58      0.61    555719
weighted avg       0.99      1.00      0.99    555719

-------------
Model saved as xgb_model.pkl


In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42)
evaluate_and_save_model(dt_classifier, X_train, X_test, y_train, y_test, 'dt_classifier')

DecisionTreeClassifier Accuracy: 0.9839
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    553574
           1       0.07      0.27      0.11      2145

    accuracy                           0.98    555719
   macro avg       0.53      0.63      0.55    555719
weighted avg       0.99      0.98      0.99    555719

-------------
Model saved as dt_classifier


In [ ]:
nb_model = GaussianNB()
evaluate_and_save_model(nb_model, X_train, X_test, y_train, y_test, 'nb_model.pkl')

GaussianNB Accuracy: 0.9907
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    553574
           1       0.20      0.46      0.28      2145

    accuracy                           0.99    555719
   macro avg       0.60      0.73      0.64    555719
weighted avg       0.99      0.99      0.99    555719

-------------
Model saved as nb_model.pkl


In [11]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
evaluate_and_save_model(nb_model, X_train_resampled, X_test, y_train_resampled, y_test, 'nb_model_smote.pkl')

GaussianNB Accuracy: 0.9762
              precision    recall  f1-score   support

           0       1.00      0.98      0.99    553574
           1       0.08      0.51      0.14      2145

    accuracy                           0.98    555719
   macro avg       0.54      0.74      0.56    555719
weighted avg       0.99      0.98      0.98    555719

-------------
Model saved as nb_model_smote.pkl


In [ ]:
evaluate_and_save_model(xgb_model, X_train_resampled, X_test, y_train_resampled, y_test, 'xgb_model_smote.pkl')

XGBClassifier Accuracy: 0.9946
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.31      0.32      0.31      2145

    accuracy                           0.99    555719
   macro avg       0.65      0.66      0.66    555719
weighted avg       0.99      0.99      0.99    555719

-------------
Model saved as xgb_model_smote.pkl
